# popDMS

**A population genetics-based method for inferring mutation effects from deep mutational scanning (DMS) data**

popDMS takes input codon counts in [dms_tools format](https://jbloomlab.github.io/dms_tools/fileformats.html#deep-mutational-scanning-counts-file) or sequence counts in [MAVE-HGVS format](https://www.mavedb.org/docs/mavehgvs/) and infers selection coefficients for each amino acid at each site in the data. Roughly, selection coefficients can be interpreted as the fractional advantage (if positive) or disadvantage (if negative) for each variant in passing through selection in the experiment.

Below, we include a basic guide for how to use the main functions of popDMS to 1) compute variant frequencies and correlations from count data, and 2) to infer selection coefficients from the computed frequencies. For more details and applications to example data sets, see [the repository](https://github.com/bartonlab/paper-DMS-inference) for the paper describing popDMS. Usage is slightly different when using codon counts and when using sequence counts. We also show how to visualize the inferred selection coefficients with a heatmap.

## Example using codon counts

Let's assume that we have a DMS data set that we refer to as `example`. In this data set, there were two experimental replicates. Codon counts before selection were saved in two files, `example_mutDNA_1.csv` and `example_mutDNA_2.csv` for replicates 1 and 2, respectively. Codon counts after selection, which here we will assume is after one generation of replication, are stored in files `example_mutvirus_1.csv` and `example_mutvirus_2.csv`. The codon counts should be in [dms_tools format](https://jbloomlab.github.io/dms_tools/fileformats.html#deep-mutational-scanning-counts-file) and should be readable by `pandas`. 

The code block below would analyze this data set and store the inferred selection coefficients in a compressed CSV file `example_selection_coefficients.csv.gz`. This file can either be analyzed by `pandas` or similar packages while compressed, or it can be uncompressed to be viewed directly. 

In [ ]:
import popDMS


#___ popDMS parameters

## `corr_cutoff_pct` sets the maximum fraction of drop in correlation per log10(gamma) for setting optimal regularization strength 
## 0.1 is a good default value
corr_cutoff_pct = 0.1  


#___ data parameters

## `name` gives the name of the data set, which will be prepended to the files that popDMS saves
name = 'example'

## `n_replicates` specifies the number of experimental replicates for this data set
n_replicates = 2

## `codon_counts_files` specifies the path to the codon counts files
## the order of the files is arbitrary, but subsequent variables (`replicates`, `times`) will assume the same order
codon_counts_files = ['example_mutDNA_1.csv',    
                      'example_mutDNA_2.csv',    
                      'example_mutvirus_1.csv',
                      'example_mutvirus_2.csv']

## `replicates` stores which replicate each file in the list above corresponds to
replicates = [1,  # replicate number for 'example_mutDNA_1.csv'
              2,  # replicate number for 'example_mutDNA_2.csv'
              1,  # replicate number for 'example_mutvirus_1.csv'
              2]  # replicate number for 'example_mutvirus_2.csv'

## `times` stores the collection time for each codon counts file
## here we set the counts before selection as generation 0 and the counts after selection as generation 1
## only differences in time are important -- setting the generations before and after selection as 1 and 2
## would yield exactly the same results, for example
times = [0,  # sample generation for 'example_mutDNA_1.csv'
         0,  # sample generation for 'example_mutDNA_2.csv'
         1,  # sample generation for 'example_mutvirus_1.csv'
         1]  # sample generation for 'example_mutvirus_2.csv'

## `freq_dir` specifies the directory where frequencies should be saved
## to save in the current directory, use '.' or leave this variable out when calling the function
freq_dir = '.'

## `output_dir` specifies the directory where selection coefficients should be saved
## to save in the current directory, use '.' or leave this variable out when calling the function
output_dir = '.'

## `plot_gamma` specifies whether or not to show a diagnostic plot of the correlation between
## replicates as a function of the regularization strength `gamma` (set to True by default)
plot_gamma = True


#___ running popDMS

# COMPUTE FREQUENCIES -- for codon counts, this should be very fast
popDMS.compute_variant_frequencies_Bloom(name = name, 
                                         codon_counts_files = codon_counts_files,
                                         replicates = replicates,
                                         freq_dir = freq_dir)


# INFER SELECTION COEFFICIENTS -- this should also be very fast for codon counts
popDMS.infer_independent(name = name, 
                         n_replicates = n_replicates, 
                         corr_cutoff_pct = corr_cutoff_pct, 
                         freq_dir = freq_dir, 
                         output_dir = output_dir, 
                         plot_gamma = plot_gamma)

## Example using sequence counts

Let's assume that we have a DMS data set that we refer to as `example`. In this data set, there were two experimental replicates. Sequence counts in [MAVE-HGVS format](https://www.mavedb.org/docs/mavehgvs/) were saved in the file `example_nucleotide_counts.csv`, which should be readable by `pandas`. Here we'll assume that the sequence counts before selection for the two replicates were stored in columns `replicate_A_0` and `replicate_B_0`, respectively. We'll assume that selection counts after selection, which we take to be 10 generations of replication, are stored in the columns `replicate_A_10` and `replicate_B_10`, respectively, in the same file. In addition to this data, we'll also need to save the nucleotide reference sequence in plain text, which here we assume is in a file `example_reference_sequence.dat`.

The code block below would analyze this data set and store the inferred selection coefficients in a compressed CSV file `example_selection_coefficients.csv.gz`. This file can either be analyzed by `pandas` or similar packages while compressed, or it can be uncompressed to be viewed directly. 

In [ ]:
import popDMS


#___ popDMS parameters

## `corr_cutoff_pct` sets the maximum fraction of drop in correlation per log10(gamma) for setting optimal regularization strength 
## 0.1 is a good default value
corr_cutoff_pct = 0.1  


#___ data parameters

## `name` gives the name of the data set, which will be prepended to the files that popDMS saves
name = 'example'

## `n_replicates` specifies the number of experimental replicates for this data set
n_replicates = 2

## `reference_sequence_file` specifies the path to the reference sequence
reference_sequence_file = 'example_reference_sequence.dat'

## `haplotype_counts_file` gives the path to the file that stores the sequence counts
haplotype_counts_file = 'example_nucleotide_counts.csv'

## `time_points` specifies the times (in generations) at which data was collected
## here we'll set the initial time as generation 0, and the second collection time after selection
## as generation 10, but note that only the differences in time are important
## setting these values to 10 and 20, respectively, would yield identical results
time_points = [0, 10]

## `time_cols` specifies the columns that store sequence counts sampled at the time above for each replicate
time_cols = {1: ['replicate_A_0', 'replicate_A_10'],  # for the 1st replicate, counts stored in columns 'replicate_A_0'/'replicate_A_10' before/after selection
             2: ['replicate_B_0', 'replicate_B_10']}  # for the 2nd replicate, counts stored in columns 'replicate_B_0'/'replicate_B_10' before/after selection

## `freq_dir` specifies the directory where frequencies should be saved
## to save in the current directory, use '.' or leave this variable out when calling the function
freq_dir = '.'

## `output_dir` specifies the directory where selection coefficients should be saved
## to save in the current directory, use '.' or leave this variable out when calling the function
output_dir = '.'

## `plot_gamma` specifies whether or not to show a diagnostic plot of the correlation between
## replicates as a function of the regularization strength `gamma` (set to True by default)
plot_gamma = True


#___ running popDMS

# COMPUTE FREQUENCIES -- for sequence counts, this can be time-consuming depending on the size of the sequences,
# the number of replicates, sequencing depth, and the number of time points included in the analysis
# see examples at https://github.com/bartonlab/paper-DMS-inference for an idea of typical times
popDMS.compute_variant_frequencies(name = name, 
                                   reference_sequence_file = reference_sequence_file, 
                                   haplotype_counts_file = haplotype_counts_file, 
                                   n_replicates = n_replicates, 
                                   time_points = time_points, 
                                   time_cols = time_cols, 
                                   freq_dir = freq_dir)


# INFER SELECTION COEFFICIENTS -- typically, this step is relatively faster than computing frequencies above
popDMS.infer_correlated(name = name, 
                        n_replicates = n_replicates, 
                        corr_cutoff_pct = corr_cutoff_pct, 
                        freq_dir = freq_dir, 
                        output_dir = output_dir, 
                        plot_gamma = plot_gamma)

## Example visualization of inferred selection coefficients

Regardless of the data source (codon counts or sequence counts), selection coefficients inferred by popDMS will be saved in the same format. The code block below gives a brief example of calling the built-in visualization of inferred selection coefficients, assuming that this data is stored in a file `example_selection_coefficients.csv.gz`. 

In [ ]:
import popDMS


#___ popDMS parameters

## `pop_file` specifies the path to the selection coefficients file
pop_file = 'example_selection_coefficients.csv.gz'

## `fig_file` specifies where to save the figure
fig_file = 'example_plot.pdf'


#___ running popDMS

# PLOT SELECTION COEFFICIENT HEATMAP
popDMS.fig_dms(pop_file = pop_file, 
               fig_file = fig_file)